# K-Sweep Evaluation -- Affective-RAG

Sweeps subgraph hop depth $k \in \{1,2,3,4,5\}$ using a trained GNN encoder.
Measures NDCG@10, AP@5, ADE, and knowledge-score statistics at each depth
to evaluate structural sensitivity of the graph-augmented retrieval.

### Prerequisites
| # | Requirement |
|---|-------------|
| 1 | Runtime set to **GPU** |
| 2 | GCP project with GCS bucket containing the dataset |
| 3 | Trained encoder checkpoint (`krag_encoder.pt`) to upload |

## Install & clone

In [ ]:
import subprocess, sys

def _pip(*a):
    subprocess.check_call([sys.executable, "-m", "pip", "install"] + list(a) + ["-q"])

_pip('torch-geometric')

REPO_URL = "https://github.com/Prashant002-1/Affective-RAG-Recommender-Systems.git"
!git clone {REPO_URL} ARAG

_pip("-r", "ARAG/requirements.txt")
print("Done. Restart runtime.")

### Restart the runtime
Go to **Runtime > Restart runtime**, then continue from the next cell.

## Authenticate to GCS

In [ ]:
from google.colab import auth
auth.authenticate_user()

import os
PROJECT_ID = "YOUR_PROJECT_ID"          # <-- set this
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

from google.cloud import storage
storage.Client(project=PROJECT_ID)
print(f"Authenticated: {PROJECT_ID}")


## Upload trained encoder

In [ ]:
from google.colab.files import upload
print("Upload trained encoder checkpoint (krag_encoder.pt)")
uploaded = upload()
ENCODER_PATH = list(uploaded.keys())[0]
print(f"Uploaded: {ENCODER_PATH}")

## Imports

In [ ]:
import sys, torch, json, numpy as np
sys.path.insert(0, "ARAG/src")

from krag.system               import ARAGSystem, ARAGSystemConfig
from krag.training.gnn_trainer  import prepare_emotion_ground_truth
from krag.data.adapters         import get_adapter
from krag.evaluation.synthetic_testset import SyntheticTestSetGenerator
from krag.evaluation.metrics    import (
    compute_ndcg,
    compute_affective_precision_at_k,
    compute_affective_displacement_error,
)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Imports OK | device = {DEVICE}")


## Initialize system

In [ ]:
config = ARAGSystemConfig()
config.vertex_ai_project = PROJECT_ID
system = ARAGSystem(config)
system.initialize()
stats  = system.load_and_index_data()
print(f"Content items : {stats['content_items']}")
print(f"KG nodes      : {stats['knowledge_graph_nodes']}")
print(f"KG edges      : {stats['knowledge_graph_edges']}")


## Load new encoder

In [ ]:
import types

system.krag_encoder.load_state_dict(
    torch.load(ENCODER_PATH, map_location=DEVICE)
)
system.krag_encoder.to(DEVICE)
system.krag_encoder.eval()

system.response_generator.generate_response = lambda *a, **kw: ""

def _index_subgraphs(self, content_ids, hops=2):
    print(f"Indexing subgraphs for {len(content_ids)} items...")
    self.encoder.eval()
    device = next(self.encoder.parameters()).device
    with torch.no_grad():
        for content_id in content_ids:
            if content_id in self.subgraph_embeddings:
                continue
            subgraph = self.kg.extract_subgraph(content_id, hops=hops)
            if len(subgraph.nodes()) > 0:
                pyg_data = self.kg.to_pytorch_geometric(subgraph, self.node_embeddings).to(device)
                embedding = self.encoder.index_subgraph(pyg_data)
                self.subgraph_embeddings[content_id] = embedding.cpu().numpy()

system.subgraph_retriever.index_subgraphs = types.MethodType(_index_subgraphs, system.subgraph_retriever)

print("Encoder loaded.")

## Load affective signatures & generate test queries

In [ ]:
adapter = get_adapter()
movies_df = adapter.load_movies(vector_ready=True)
aff_sigs  = prepare_emotion_ground_truth(movies_df)   # {id: np.array(6)}
content_ids = [str(item.id) for item in system.content_items]
print(f"Affective signatures : {len(aff_sigs)}")

_result = system.vector_store.semantic_collection.get(include=["embeddings"])
_emb_map = dict(zip(_result["ids"], _result["embeddings"]))
content_embeddings = np.array([_emb_map[cid] for cid in content_ids])
print(f"Content embeddings   : {content_embeddings.shape}")

gen = SyntheticTestSetGenerator(
    content_embedder=system.content_embedder,
    semantic_threshold=0.5,
    affective_threshold=0.6,
    seed=42,
)
test_cases = gen.generate_test_set(
    content_items=system.content_items,
    content_embeddings=content_embeddings,
    movie_affective_signatures=aff_sigs,
    num_queries=800,
    min_relevant=3,
)
print(f"Test queries         : {len(test_cases)}")

## Run k-sweep

For each depth the subgraph index is rebuilt and the full test set is
evaluated.  `knowledge_score` is extracted from each recommendation to
verify the graph signal is no longer flat at 0.5.


In [ ]:
EMOTION_ORDER = ["happiness", "sadness", "anger", "fear", "surprise", "disgust"]
K_VALUES      = [1, 2, 3, 4, 5]
sweep         = {}

for k in K_VALUES:
    print(f"\n--- k = {k} ---", flush=True)
    system.subgraph_retriever.subgraph_embeddings = {}
    system.subgraph_retriever.index_subgraphs(content_ids, hops=k)

    ndcg_list, ap5_list, ade_list, ks_list = [], [], [], []

    for tc in test_cases:
        if isinstance(tc.target_emotions, dict):
            target_vec  = np.array([tc.target_emotions.get(e, 0.0) for e in EMOTION_ORDER])
            sliders     = {e: int(round(v * 10)) for e, v in tc.target_emotions.items()}
        else:
            target_vec  = np.asarray(tc.target_emotions)
            sliders     = {e: int(round(v * 10)) for e, v in zip(EMOTION_ORDER, target_vec.tolist())}

        recs  = system.query(
            query_text=tc.query_text, emotion_sliders=sliders,
            max_results=10, include_explanation=False
        )
        items = recs.get("recommendations", [])
        if not items:
            continue

        retrieved_ids = [r["content_id"] for r in items]

        ndcg_list.append(compute_ndcg(retrieved_ids, tc.relevant_items, k=10))
        ap5_list.append(compute_affective_precision_at_k(
            retrieved_ids, aff_sigs, target_vec, 5))

        ade = compute_affective_displacement_error(
            retrieved_ids, aff_sigs, target_vec, 10)
        if ade != float('inf'):
            ade_list.append(ade)

        for r in items:
            ks_list.append(r["scores"]["knowledge"])

    sweep[k] = {
        "ndcg_10":              float(np.mean(ndcg_list)) if ndcg_list else 0.0,
        "ap_5":                 float(np.mean(ap5_list))  if ap5_list  else 0.0,
        "ade":                  float(np.mean(ade_list))  if ade_list  else float('inf'),
        "knowledge_score_mean": float(np.mean(ks_list))   if ks_list   else 0.0,
        "knowledge_score_std":  float(np.std(ks_list))    if ks_list   else 0.0,
        "n_queries":            len(ndcg_list),
    }
    r = sweep[k]
    print(f"  NDCG@10  {r['ndcg_10']:.4f}   AP@5  {r['ap_5']:.4f}   "
          f"ADE  {r['ade']:.4f}   KS  {r['knowledge_score_mean']:.4f} +/- {r['knowledge_score_std']:.4f}")

print("\nSweep complete.")

## Results table

In [ ]:
print(f"{'k':>3} | {'NDCG@10':>8} | {'AP@5':>6} | {'ADE':>6} | {'KS mean':>8} | {'KS std':>7} | {'n':>4}")
print("-" * 66)
for k in K_VALUES:
    r = sweep[k]
    print(f"{k:>3} | {r['ndcg_10']:>8.4f} | {r['ap_5']:>6.4f} | {r['ade']:>6.4f} | "
          f"{r['knowledge_score_mean']:>8.4f} | {r['knowledge_score_std']:>7.4f} | {r['n_queries']:>4}")


## Plots

In [ ]:
import matplotlib.pyplot as plt

ks      = list(sweep.keys())
ndcgs   = [sweep[k]['ndcg_10']              for k in ks]
ks_mean = [sweep[k]['knowledge_score_mean'] for k in ks]
ks_std  = [sweep[k]['knowledge_score_std']  for k in ks]
ap5s    = [sweep[k]['ap_5']                 for k in ks]
ades    = [sweep[k]['ade']                  for k in ks]

plot_cfg = {
    "ndcg": ("Hop depth k", "NDCG@10", "Ranking quality vs depth", ndcgs, None, "steelblue"),
    "ks":   ("Hop depth k", "Knowledge score", "Graph signal (mean +/- std)", ks_mean, ks_std, "darkorange"),
    "ap5":  ("Hop depth k", "AP@5", "Affective precision vs depth", ap5s, None, "seagreen"),
    "ade":  ("Hop depth k", "ADE (lower is better)", "Displacement error vs depth", ades, None, "firebrick"),
}

for name, (xlabel, ylabel, title, vals, errs, color) in plot_cfg.items():
    fig, ax = plt.subplots(figsize=(5, 3.5))
    if errs is not None:
        ax.errorbar(ks, vals, yerr=errs, marker='o', color=color, capsize=4, linewidth=2)
        ax.axhline(0.5, color='gray', linestyle='--', alpha=0.6, label='Uniform baseline')
        ax.legend()
    else:
        ax.plot(ks, vals, marker='o', color=color, linewidth=2)
    ax.set_xlabel(xlabel); ax.set_ylabel(ylabel); ax.set_title(title)
    ax.set_xticks(ks); ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"/tmp/k_sweep_{name}.png", dpi=200, bbox_inches="tight")
    plt.show()

print("Saved 4 individual plots.")

## Save & download

In [ ]:
out = {
    "metadata": {
        "experiment": "k_sweep",
        "encoder": "krag_encoder_aw0.3",
        "encoder_config": {
            "contrastive_weight": 0.2,
            "alignment_weight": 0.3,
            "embedding_dim": 768,
            "num_layers": 3,
            "num_heads": 4,
        },
        "k_values": K_VALUES,
        "n_queries": len(test_cases),
        "device": DEVICE,
    },
    "results": {str(k): v for k, v in sweep.items()},
}
with open("/tmp/k_sweep_results.json", "w") as f:
    json.dump(out, f, indent=2)

from google.colab.files import download
download("/tmp/k_sweep_results.json")
for name in ["ndcg", "ks", "ap5", "ade"]:
    download(f"/tmp/k_sweep_{name}.png")
print("Downloads triggered.")